# Overview
In this notebook I am going to describe how in general R-CNN and its children (Fast R-CNN and Faster R-CNN) work.
My describtion will be based on part of Udacity Computer Vision Nanodegree and those blogs: [one](https://towardsdatascience.com/deep-learning-for-object-detection-a-comprehensive-review-73930816d8d9), [two](https://www.learnopencv.com/selective-search-for-object-detection-cpp-python/).

### R-CNN

Region-based Convolutional Neural Network consists of 2 main steps:
#### 1. Region proposal algorithm is used to generate many areas where objects may be found
<img src="images/RegionProposal.png"><br>
Credits: Udacity Computer Vision Nanodegree<br><br>
**Q1**: But what is and how does **region proposal algorithm** work?<br>
**A1**: These methods takes image as an input and then output bounding boxes(coordinates) of all chunks of image which has high chance to be an object. Having extracted those areas we can then perform iamge recognition to determine what excacly that object is. This family of algorithms identify objects through **segmentation**. In segmentation, adjacent regions of image similar to each other (based on some criteria like color, gradient, texture etc.) are grouped. Also, those algorithms have a very high **recall** which means that we have many false positives detected (background), but almost for sure we have detected all true positives. In this case having many false positves is not a problem because they are going to be rejected during obejct recognition phase.<br>
**Example of such algorithm is Selective Search described comprehensively in blog [two](https://www.learnopencv.com/selective-search-for-object-detection-cpp-python/)**.<br><br>
#### 2. Each of proposed areas(region of interest = RoI) goes through classification CNN which outputs a class label and bounding box
<img src="images/RoItoCNN.png"><br>
Credits: Udacity Computer Vision Nanodegree<br><br>
Every RoI goes through CNN and the class label is predicted (for example DOG). In this case, class **backgroud** is included in order to capture any noisy regions. During training such model loss function is a weighted average of loss for classification (predicting classes) and regression(predicting coordinates for bounding boxes) problems.<br><br>

**Q2**: What was the biggest disadvantage of that method?<br>
**A2**: When processing one image, CNN has to be run on every RoI which drasticly increases time of both training and detecting.


### Fast R-CNN
R-CNN is accurate but quite slow and inefficient approach. In order to address that **Fast R-CNN** came. The main difference is that Fast R-CNN runs entire image through the CNN only once!<br>
**Q3**: But how is this possible?<br>
**A3**: RoI pooling layers were introduced. Their task is to take as an input feature map of given image, and then for that feature map for every RoI then perform max pooling in such a way that the output matches some pre-defined value (eg. 2x2)
<img src="https://cdn-sv1.deepsense.ai/wp-content/uploads/2017/02/roi_pooling-1.gif"><br>
Credits: [deepsense](https://deepsense.ai/region-of-interest-pooling-explained/)<br><br>

This approach is much faster then R-CNN but still uses region proposal algortihm for detecting potencial objects.

### Faster R-CNN
Here we finally replace slow region proposal algorithm(eg. selective search) with **region proposal network**.<br>
**Q4**: What is RPN?<br>
**A4**: RPN like the name says, generates proposal for the objects.<br>
<img src="https://cdn-images-1.medium.com/max/1000/1*7heX-no7cdqllky-GwGBfQ.png"><br>
Credits: [blog one](https://towardsdatascience.com/deep-learning-for-object-detection-a-comprehensive-review-73930816d8d9)<br><br>
Great explanation can be found [here](https://www.quora.com/How-does-the-region-proposal-network-RPN-in-Faster-R-CNN-work).

# TO DO

- Expand Faster R-CNN explanation above using this [link](https://www.quora.com/How-does-the-region-proposal-network-RPN-in-Faster-R-CNN-work) (easy) 
- Go through [that](http://www.telesens.co/2018/03/11/object-detection-and-classification-using-r-cnns/) blog post with very detailed tutorial how to implement Faster R-CNN in PyTorch (rather hard and time-consuming but very rewarding probably)